![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_AlbertForMultipleChoice..ipynb)

# Import OpenVINO AlbertForMultipleChoice models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting AlbertForMultipleChoice  models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for AlbertForMultipleChoice from ALBERT  and they have to be in `For Multiple Choice` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.41.2`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install --quiet onnx==1.18.0 onnxruntime==1.22.0 optimum[onnxruntime]==1.26.1 openvino==2025.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- Normally, to load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. However, ForMultipleChoice is not yet available so we will use `openvino.convert_model()` after exporting ONNX model
- We'll use [Ariffiq99/CRAB_COPA_KUCI_e_care_albert_Base_Finetuned](https://huggingface.co/Ariffiq99/CRAB_COPA_KUCI_e_care_albert_Base_Finetuned) model from HuggingFace as an example
- In addition to the Albert model, we also need to save the `AutoTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from optimum.onnxruntime import ORTModelForMultipleChoice

MODEL_NAME = "Ariffiq99/CRAB_COPA_KUCI_e_care_albert_Base_Finetuned"
ONNX_MODEL_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForMultipleChoice.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(ONNX_MODEL_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/866 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.7M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -lh {ONNX_MODEL_PATH}

total 45M
-rw-r--r-- 1 root root 795 Jun 27 04:41 config.json
-rw-r--r-- 1 root root 45M Jun 27 04:41 model.onnx


In [4]:
from transformers import AutoTokenizer

OPENVINO_MODEL_PATH = "./openvino_models"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(OPENVINO_MODEL_PATH)

('./openvino_models/tokenizer_config.json',
 './openvino_models/special_tokens_map.json',
 './openvino_models/spiece.model',
 './openvino_models/added_tokens.json',
 './openvino_models/tokenizer.json')

In [5]:
import openvino as ov

ov_model = ov.convert_model(f"{ONNX_MODEL_PATH}/model.onnx")
ov.save_model(ov_model, f"{OPENVINO_MODEL_PATH}/openvino_model.xml")

In [6]:
!ls -lh {OPENVINO_MODEL_PATH}

total 26M
-rw-r--r-- 1 root root  23M Jun 27 04:41 openvino_model.bin
-rw-r--r-- 1 root root 613K Jun 27 04:41 openvino_model.xml
-rw-r--r-- 1 root root  970 Jun 27 04:41 special_tokens_map.json
-rw-r--r-- 1 root root 743K Jun 27 04:41 spiece.model
-rw-r--r-- 1 root root 1.5K Jun 27 04:41 tokenizer_config.json
-rw-r--r-- 1 root root 2.2M Jun 27 04:41 tokenizer.json


In [7]:
!mkdir {OPENVINO_MODEL_PATH}/assets && mv {OPENVINO_MODEL_PATH}/spiece.model {OPENVINO_MODEL_PATH}/assets

In [9]:
!ls -lh {OPENVINO_MODEL_PATH}/assets

total 744K
-rw-r--r-- 1 root root 743K Jun 27 04:41 spiece.model


## Import and Save AlbertForMultipleChoice in Spark NLP


- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [1]:
!pip install -q spark-nlp==6.0.3 pyspark==3.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.0/713.0 kB 1.1 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [2]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  6.0.3
Apache Spark version:  3.5.1


- Let's use `loadSavedModel` functon in `AlbertForMultipleChoice` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `AlbertForMultipleChoice` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [4]:
from sparknlp.annotator import AlbertForMultipleChoice

albertMultpleChoiceClassifier = AlbertForMultipleChoice.loadSavedModel(
     f"{OPENVINO_MODEL_PATH}",
     spark
 )\
  .setInputCols(["document_question", "document_context"])\
  .setOutputCol("answer")

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [5]:
albertMultpleChoiceClassifier.write().overwrite().save("./{}_spark_nlp_openvino".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [6]:
!rm -rf {ONNX_MODEL_PATH}
!rm -rf {OPENVINO_MODEL_PATH}

Awesome 😎  !

This is your`AlbertForMultipleChoice` model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [7]:
! ls -l {MODEL_NAME}_spark_nlp_openvino

total 24192
-rw-r--r-- 1 root root 24004008 Jun 27 04:50 albert_mc_classification_openvino
-rw-r--r-- 1 root root   760289 Jun 27 04:50 albert_spp
drwxr-xr-x 2 root root     4096 Jun 27 04:50 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny `AlbertForMultipleChoice` model 😊

In [10]:
data = spark.createDataFrame([
    ("In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.", "It is eaten with a fork and a knife, It is eaten while held in the hand."),
    ("The Eiffel Tower is located in which country?", "Germany, France, Italy"),
    ("Which animal is known as the king of the jungle?", "Lion, Elephant, Tiger, Leopard"),
    ("Water boils at what temperature?", "90°C, 120°C, 100°C"),
    ("Which planet is known as the Red Planet?", "Jupiter, Mars, Venus"),
    ("Which language is primarily spoken in Brazil?", "Spanish, Portuguese, English"),
    ("The Great Wall of China was built to protect against invasions from which group?", "The Greeks, The Romans, The Mongols, The Persians"),
    ("Which chemical element has the symbol 'O'?", "Oxygenm, Osmium, Ozone"),
    ("Which continent is the Sahara Desert located in?", "Asia, Africa, South America"),
    ("Which artist painted the Mona Lisa?", "Vincent van Gogh, Leonardo da Vinci, Pablo Picasso")
], ["question", "choices"])


This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [12]:
from sparknlp.base import MultiDocumentAssembler
from sparknlp.annotator import AlbertForMultipleChoice
from pyspark.ml import Pipeline

document_assembler = MultiDocumentAssembler()\
    .setInputCols(["question", "choices"])\
    .setOutputCols(["document_question", "document_choices"])

model = AlbertForMultipleChoice()\
    .load(f"{MODEL_NAME}_spark_nlp_openvino")\
    .setInputCols(["document_question", "document_choices"])\
    .setOutputCol("answer")\
    .setBatchSize(4)

pipeline = Pipeline(stages=[
    document_assembler,
    model
])

model = pipeline.fit(data)
result = model.transform(data)

result.selectExpr("question", "choices", "answer.result[0] as predicted_answer").show(truncate=False)

+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+-----------------------------------+
|question                                                                                  |choices                                                                 |predicted_answer                   |
+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+-----------------------------------+
|In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.|It is eaten with a fork and a knife, It is eaten while held in the hand.|It is eaten with a fork and a knife|
|The Eiffel Tower is located in which country?                                             |Germany, France, Italy                                                  | France                    

That's it! You can now go wild and use hundreds of `AlbertForMultipleChoice` models from HuggingFace 🤗 in Spark NLP 🚀
